In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.applications import DenseNet201
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.utils import to_categorical
import os
import cv2

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [39]:
[2]*3

[2, 2, 2]

In [43]:
X_train = []
y_train = []
X_test = []
y_test = []

# Expiro: 0
X_train += load_images_from_folder('malevis_train_val_224x224/train/Expiro')
y_train += [0]*350
X_test += load_images_from_folder('malevis_train_val_224x224/val/Expiro')
y_test += [0]*151

# Neshta: 1
X_train += load_images_from_folder('malevis_train_val_224x224/train/Neshta')
y_train += [1]*350
X_test += load_images_from_folder('malevis_train_val_224x224/val/Neshta')
y_test += [1]*147

# Sality: 1
X_train += load_images_from_folder('malevis_train_val_224x224/train/Sality')
y_train += [2]*350
X_test += load_images_from_folder('malevis_train_val_224x224/val/Sality')
y_test += [2]*149

In [44]:
X_train_array = np.array(X_train)
X_test_array = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

X_train = X_train_array.astype('float32')
X_test = X_test_array.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

X_traincv, X_testcv, y_traincv, y_testcv = train_test_split(X_train,
                                                            y_train,
                                                            test_size=0.3)

In [48]:
nn = DenseNet201(
    include_top=True,
    weights=None,
    classes=3
)

nn.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

stats_accu = []
stats_loss = []

batch_size = 64
epochs = 60

class NBatchLogger(Callback):
    "A Logger that log average performance per `display` steps."

    def __init__(self, display=100):
        self.step = 0
        self.display = display
        self.metric_cache = {}

    def on_train_batch_end(self, batch, logs={}):
        self.step += 1
        #print(logs)
        stats_accu.append(logs['accuracy'])
        stats_loss.append(logs['loss'])
        #for k in self.params['metrics']:
         #   print(k)

In [ ]:
nn_train = nn.fit(X_traincv, y_traincv, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(X_testcv, y_testcv), callbacks=[NBatchLogger()])

Epoch 1/60
